Before we start writing queries with PARTITION BY ORDER BY, let's quickly revise queries with PARTITION BY alone. Take a look:
```
SELECT
  country,
  city,
  rating,
  AVG(rating) OVER(PARTITION BY country)
FROM store;
```

For each sales row, show the store_id, day, revenue on that day and the average revenue in that store.
```
select 
	store_id, day, revenue,
    AVG(revenue) OVER(partition by store_id)
from sales;
```

For each sales row between August 1 and August 7, 2016, show the store_id, day, number of transactions, the total number of transactions on that day in any store and the ratio of the two last columns shown as percentage rounded to integer values.

```
select 
	store_id, day, transactions,
    sum(transactions) over(partition by day),
    ROUND(100 * transactions::numeric / sum(transactions) over(partition by day))
from sales
where day >= '2016-08-01' 
and day <= '2016-08-07'
```

Now, we can add PARTITION BY to calculate the positions independently for each country:
```
SELECT
  id,
  country,
  city,
  rating,
  RANK() OVER(PARTITION BY country ORDER BY rating DESC)
FROM store;
```
In this way, we create a separate ranking for each country, so Paris and Frankfurt can both get rank = 1 for the separate rankings in France and Germany:

Take into account the period between August 10 and August 14, 2016. For each row of sales, show the following information: store_id, day, number of customers and the rank based on the number of customers in the particular store (in descending order).
```
select
	store_id,day,customers,
    rank() over(partition by store_id order by customers desc)
from sales
where day >= '2016-08-10' and day <= '2016-09-14'
```

Good job! Of course, you can use any other ranking function in the same way:
```
SELECT
  id,
  country,
  city,
  rating,
  NTILE(2) OVER(PARTITION BY country ORDER BY opening_day)
FROM store;
```
In the above query, the stores are divided into two groups: older and more recent stores. These groups are created separately for each country.

Take the sales between August 1 and August 10, 2016. For each row, show the store_id, the day, the revenue on that day and quartile number (quartile means we divide the rows into four groups) based on the revenue of the given store in the descending order.
```
select
    store_id,day,revenue,
    NTILE(4) over(partition by store_id order by revenue desc)
from sales
where day >= '2016-08-01' and day <= '2016-08-10'
```

Alright! Do you remember queries that introduced WITH? We used them to find the row with a certain rank. Now, we can find even more rows with a certain rank, each for a different group. Take a look:
```
WITH ranking AS (
  SELECT
    country,
    city,
    RANK() OVER(PARTITION BY country ORDER BY rating DESC) AS rank
  FROM store
)

SELECT
  country,
  city
FROM ranking
WHERE rank = 1;
```
The CTE in the parentheses creates a separate ranking of stores in each country based on their rating. In the outer query, we simply return the rows with the right rank. As a result, we'll see the best store in each country.

For each store, show a row with three columns: `store_id`, the `revenue` on the best day in that store in terms of the revenue and the day when that best revenue was achieved.
```
WITH ranking AS (
  SELECT
    store_id,
    revenue,
  	day,
    RANK() OVER(PARTITION BY store_id ORDER BY revenue DESC) AS rank
  FROM sales
)

SELECT store_id, revenue, day
FROM ranking
WHERE rank = 1;
```

Let's analyze sales data between August 1 and August 3, 2016. For each row, show store_id, day, transactions and the ranking of the store on that day in terms of the number of transactions as compared to other stores. The store with the greatest number should get 1 from a window function. Use individual row ranks even when two rows share the same value. Name the column place_no.

```
select
	store_id,
    day,
    transactions,
    ROW_NUMBER() over(partition by day order by transactions desc) as place_no
from sales
where day >= '2016-08-01' and day <= '2016-08-03'
```

For each day of the sales statistics, show the day, the store_id of the best store in terms of the revenue on that day, and that revenue.

```
with ranking as (select
	day, store_id, revenue,
    RANK() over(partition by day order by revenue desc)
from sales)

select day, store_id, revenue from ranking where rank = 1;
```

Divide the sales results for each store into four groups based on the number of transactions and for each store, show the rows in the group with the lowest numbers of transactions: store_id, day, transactions.
```
with ranking as (select
	store_id, day, transactions,
    NTILE(4) over(partition by store_id order by transactions)
from sales)

select store_id, day, transactions from ranking where ntile = 1; 
```

Alright! Let's move on. In part 5, you got to know window frames. Can we use them together with PARTITION BY to create even more sophisticated windows? Of course we can. Take a look:
```
SELECT
  id,
  country,
  city,
  opening_day,
  rating,
  MAX(rating) OVER(
    PARTITION BY country
    ORDER BY opening_day
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM store;
```
In the above example, we show some information about each store and the maximal rating of any store opened up to that date (that's where we need a window frame) in the respective country (that's where we need PARTITION BY).

Show sales statistics between August 1 and August 7, 2016. For each row, show store_id, day, revenue and the best revenue in the respective store up to that date.
```
select
	store_id, day, revenue,
    MAX(revenue) over(partition by store_id order by day rows between unbounded preceding and current row)
from sales
where day >= '2016-08-01' 
and day <= '2016-08-07'
```

Take sales from the period between August 1 and August 10, 2016. For each row, show the following information: store_id, day, number of transactions and the average number of transactions in the respective store in the window frame starting 2 days before and ending 2 days later with respect to the current row.
```
select
	store_id, day, transactions,
    AVG(transactions) over(partition by store_id order by day rows between 2 preceding and 2 following)
from sales
where day >= '2016-08-01' 
and day <= '2016-08-10'
```

For each sales row, show the following information: store_id, day, revenue and the future cash flow receivable by the headquarters (i.e. the total revenue in that store, counted from the current day until the last day in our table).
```
select
	store_id, day, revenue,
    sum(revenue) over(partition by store_id order by day ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING)
from sales
```

Now, let's talk about the use of analytical functions with PARTITION BY ORDER BY. Take a look at the following example:
```
SELECT
  country,
  city,
  opening_day,
  LEAD(city, 1, 'NaN') OVER(PARTITION BY country ORDER BY opening_day)
FROM store;
```
In the above example, we show the country, city and opening_day of each store, but we also show the city where the next store was opened – in the same country, of course.

For each store, show the sales in the period between August 5, 2016 and August 10, 2016: store_id, day, number of transactions, number of transactions on the previous day and the difference between these two values.
```
SELECT
  store_id,
  day,
  transactions,
  lag(transactions) OVER(PARTITION BY store_id ORDER BY day),
  transactions - LEAD(transactions, 1) OVER(PARTITION BY store_id ORDER BY day desc)
FROM sales
where day >= '2016-08-05' 
and day <= '2016-08-10'
;
```

Good job! Of course, other analytical functions are possible as well. Let's analyze another example:
```
SELECT
  country,
  city,
  rating,
  FIRST_VALUE(city) OVER(PARTITION BY country ORDER BY rating DESC)
FROM store;
```
In the above query, we're showing each store individually, but we also show the name of the city with the highest rating in that particular country. Note that this would be impossible without PARTITION BY – we couldn't get individual city names for each country separately.

Show sales figures in the period between August 1 and August 3: for each store, show the store_id, the day, the revenue and the date with the best revenue in that period as best_revenue_day.
```
select 
	store_id, day, revenue,
    FIRST_VALUE(day) over(partition by store_id order by revenue desc) as best_revenue_day
FROM sales
where day >= '2016-08-01' 
and day <= '2016-08-03'
```

For each row of the sales figures, show the following information: store_id, day, revenue, revenue a week before and the ratio of revenue today to the revenue a week before expressed in percentage with 2 decimal places.
```
select
	store_id, day, revenue,
    lag(revenue, 7) over(partition by store_id order by day rows between unbounded preceding and current row),
    ROUND(100 * revenue::numeric / lag(revenue, 7) over(partition by store_id order by day rows between unbounded preceding and current row), 2)
from sales
```

For each row, show the following columns: store_id, day, customers and the number of clients in the 5th greatest store in terms of the number of customers on that day.
```
select
	store_id, day, customers,
    NTH_VALUE(customers, 5) over(partition by day order by customers desc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
from sales
```

For each day, show the following two columns: day and the name of the second most frequently repaired phone on that day. Only take into account free_repairs.
```
with ranking as (select
	day,
    phone,
    rank() over(partition by day order by free_repairs desc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
from repairs)

select day, phone from ranking where rank = 2
```

For each phone, show the following information: phone, day, revenue and the revenue for the first repair for each phone (column name first_revenue)
```
select
	phone, day, revenue,
    nth_value(revenue, 1) over(partition by phone order by day ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as first_revenue
from repairs
```

For each phone, show the following information: phone, day, the number of paid repairs, the number of paid repairs on the previous day and the difference between these two values.
```
select
	phone, day, paid_repairs,
    lag(paid_repairs, 1) over(partition by phone order by day ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING),
    paid_repairs - lag(paid_repairs, 1) over(partition by phone order by day ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
from repairs
```